### 수행단계
- 사용자별 AP@K의 평균을 계산하는 코드를 개발한다.

<!-- Precision@K와 Recall@K 정의

1. Precision@K (정밀도): 추천한 영화들 중 사용자가 실제로 평가한 영화의 비율
공식: Precision@K = TP / (TP + FP)
TP (True Positive): 추천한 영화 중 사용자가 평가한 영화
FP (False Positive): 추천한 영화 중 사용자가 평가하지 않은 영화

2. Recall@K (재현율): 사용자가 실제로 평가한 영화 중 추천된 영화의 비율
공식: Recall@K = TP / (TP + FN)
TP (True Positive): 추천한 영화 중 사용자가 평가한 영화
FN (False Negative): 사용자가 평가한 영화 중 추천되지 않은 영화 -->

In [19]:
# 평균 정밀도(AP@K)란?
# - 추천 시스템의 성능을 측정하기 위해 사용되는 지표.
# - 사용자가 실제로 좋아한(평점을 매긴) 영화들 중에서 추천된 영화들이 얼마나 잘 맞아떨어졌는지를 평가.
# 
# 평균 정밀도(AP@K) 공식
# AP@K = (1 / m) * Σ(Precision@i * rel(i)) for i = 1 to K
# - m: 사용자가 좋아한(평점을 매긴) 영화의 총 개수
# - K: 추천된 영화의 개수
# - Precision@i: i번째 영화까지의 정밀도 = TP / (i + 1)
# - rel(i): i번째 영화가 사용자가 좋아했는지 여부 (1 또는 0)
#
#
#
# 평균 정밀도 평균(MAP: Mean Average Precision)이란?
# - 여러 사용자에 대한 평균 정밀도(AP@K)의 평균 값.
# - 추천 시스템의 전체적인 성능을 평가하는 데 사용됨.
# - 각 사용자별 AP@K 값을 구한 후, 전체 사용자에 대해 평균을 내어 계산.

# 1. 추천 세트 하나 선택
#    - 예를 들어, 첫 번째 추천 세트 `["가위손", "나 홀로 집에", "대부"]` 를 선택.

# 2. 사용자 10명의 AP@K 계산 (movie_ratings_with_users.csv)
#    - 10명의 사용자 각각에 대해 AP@K를 구함.
#    - 즉, 10개의 AP@K 값이 생성됨.

# 3. 10명의 AP@K 평균을 구함 → MAP 값 생성  
#    - 첫 번째 추천 세트의 MAP 값이 계산됨.

# 4. 위 과정을 추천 세트 7개에 대해 반복 
#    - 즉, 추천 세트마다 10명의 AP@K 평균을 구해서 MAP 값을 저장.
#    - 최종적으로 7개의 MAP 값이 결과로 출력됨**.

In [3]:
import pandas as pd

# 평균 정밀도(AP@K) 계산 함수
def calculate_apk(df, recommended_movies, user_index):
    """
    사용자별 평균 정밀도(AP@K)를 계산하는 함수
    Parameters:
    - df: 사용자-영화 평가 데이터가 포함된 DataFrame
    - recommended_movies: 추천된 영화 리스트
    - user_index: 사용자 인덱스
    Returns:
    - AP@K 값 (평균 정밀도)
    """
    user_data = df.iloc[user_index, 1:].values  # 사용자 ID 제외
    recommended_data = df.loc[user_index, recommended_movies].values  # 추천된 영화 데이터 추출
    m = sum(user_data)  # 사용자가 좋아한(평점을 준) 영화의 개수
    if m == 0:
        return 0  # 좋아한 영화가 없으면 AP@K는 0
    
    apk = 0
    tp = 0  # True Positive 개수 누적
    for i in range(len(recommended_data)):
        if recommended_data[i] == 1:  # 사용자가 좋아한 영화인 경우
            tp += 1
            apk += tp / (i + 1)  # Precision@i를 누적
    return apk / m  # AP@K 계산

# MAP(Mean Average Precision) 계산 함수
def calculate_map(df, recommendation_sets):
    """
    전체 사용자에 대한 MAP(평균 AP@K)를 계산하는 함수
    Parameters:
    - df: 사용자-영화 평가 데이터가 포함된 DataFrame
    - recommendation_sets: 여러 추천 영화 리스트를 포함한 리스트
    Returns:
    - MAP 값이 포함된 DataFrame
    """
    results = []
    for rec_movies in recommendation_sets:
        all_apk = [calculate_apk(df, rec_movies, user_idx) for user_idx in range(df.shape[0])]
        mean_apk = sum(all_apk) / len(all_apk)  # 전체 사용자에 대한 AP@K 평균 계산
        results.append({"추천 영화": rec_movies, "MAP": mean_apk})
    return pd.DataFrame(results)

# 데이터 로드
csv_file_path = 'movie_ratings_with_users.csv'
df = pd.read_csv(csv_file_path)

# 추천 세트 정의
recommendation_sets = [
    ["가위손", "나 홀로 집에", "대부"],
    ["가위손", "나 홀로 집에", "대부", "사운드 오브 뮤직"],
    ["나 홀로 집에", "사운드 오브 뮤직", "아마데우스"],
    ["가위손", "대부", "터미네이터 2:오리지널"],
    ["가위손", "시네마 천국", "터미네이터 2:오리지널"],
    ["가위손", "죽은 시인의 사회", "터미네이터 2:오리지널"],
    ["아마데우스", "죽은 시인의 사회", "터미네이터 2:오리지널"]
]

# MAP 계산
map_results = calculate_map(df, recommendation_sets)

# 결과 출력
print("사용자별 추천 세트에 대한 MAP(Mean Average Precision) 결과")
display(map_results)


사용자별 추천 세트에 대한 MAP(Mean Average Precision) 결과


,추천 영화,MAP
0,"[가위손, 나 홀로 집에, 대부]",0.337619
1,"[가위손, 나 홀로 집에, 대부, 사운드 오브 뮤직]",0.415714
2,"[나 홀로 집에, 사운드 오브 뮤직, 아마데우스]",0.227222
3,"[가위손, 대부, 터미네이터 2:오리지널]",0.413492
4,"[가위손, 시네마 천국, 터미네이터 2:오리지널]",0.396825
5,"[가위손, 죽은 시인의 사회, 터미네이터 2:오리지널]",0.445714
6,"[아마데우스, 죽은 시인의 사회, 터미네이터 2:오리지널]",0.341825
